# LogisticClassification

Binary Classification : 두가지로 분류  
대표적 예 : 스펨 메일 분류, 페이스북 알고리즘, Finance 등

Linear Regression가 분류에서 가지는 한계  
* 특잇값 때문에 왜곡이 일어날 수 있음  
* 0,1 외에 너무 큰 값을 반환할 수 있음

따라서 0,1사이의 값을 주는 함수를 찾음  

 시그모이드 $$ g(z) = \frac{1}{1+ e^{-z}} $$

이제 Hypothesis를  
$ z = WX $  
$ H(x) = g(z) $  
즉, $$ H(X) = \frac{1}{1+ e^{-W^TX}} $$

기존 손실함수는  
$$ Cost = \frac{1}{m} \sum_{i=1}^m (H(x_i) - y_i)^2) $$  
이때 이를 바꾼 $H(x)$에 대해 그대로 적용하면 Convex하지 않고 울퉁불퉁한 그래프 모양이 나옴  
이때 Local minimum이 많이 생기기 때문에 Global minimun으로 학습이 되지 않음  
따라서 다른 손실 함수 필요

**New cost function for logistic**
$$ cost(W) = \frac{1}{m} \sum c(H(x),y) $$
$$c(H(x), y) = \begin{cases} 
-\log(H(x)) & y=1 \\ 
-\log(1-H(x)) & y=0 
\end{cases}$$

**해석**  
자연상수e가 함수 모양을 울퉁불퉁하게 만들기에 log를 활용하여 매끄럽게 바꿔줌  
실제레이블이 1일때
* 예측값이 1로 맞았을때는 cost값이 0  
* 예측값이 0으로 틀렸을때는 cost값이 무한히 발산  

실제레이블이 0일때
* 예측값이 0으로 맞았을때는 cost값이 0  
* 예측값이 1로 틀렸을때는 cost값이 무한히 발산

이는 case로 쪼개져 있으므로 손실함수로 프로그램에 사용하기 위해 한줄로 표현시  
$$ cost(W) = \frac{1}{m} \sum c(H(x),y) $$
$$c(H(x), y) = -y\log(H(x)) -(1-y)\log(1-H(x))$$

데이터 준비

In [4]:
import tensorflow as tf

x_data = [[1, 2],
          [2, 3],
          [3, 1],
          [4, 3],
          [5, 3],
          [6, 2]]
y_data = [[0],
          [0],
          [0],
          [1],
          [1],
          [1]]

x_data = tf.cast(x_data, tf.float32)
y_data = tf.cast(y_data, tf.float32)

가중치와 편향 만들기

In [5]:
W = tf.Variable(tf.random.normal([2, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

optimizer = tf.optimizers.SGD(learning_rate=0.01)

가설 (Hypothesis): Sigmoid 함수 적용  
$ H(x) = \frac{1}{(1 + exp^{-(Wx + b)}} $  

비용 함수 (Cost function) 적용  
$cost(H(x), y) = -y\log(H(x)) -(1-y)\log(1-H(x))$

In [6]:
for step in range(10001):
    with tf.GradientTape() as tape:
        hypothesis = tf.sigmoid(tf.matmul(x_data, W) + b)

        cost = -tf.reduce_mean(y_data * tf.math.log(hypothesis) + (1 - y_data) * tf.math.log(1 - hypothesis))

    grads = tape.gradient(cost, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))

    if step % 200 == 0:
        print(step, cost.numpy())

hypothesis = tf.sigmoid(tf.matmul(x_data, W) + b)


predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, y_data), dtype=tf.float32))

print("\nHypothesis: \n", hypothesis.numpy(), 
      "\nCorrect (Y): \n", predicted.numpy(), 
      "\nAccuracy: ", accuracy.numpy())

0 0.6118011
200 0.4231594
400 0.36791316
600 0.33932242
800 0.32137775
1000 0.3082878
1200 0.2976899
1400 0.2885197
1600 0.28025836
1800 0.27263513
2000 0.26549995
2200 0.25876418
2400 0.25237182
2600 0.24628502
2800 0.2404759
3000 0.23492311
3200 0.22960901
3400 0.22451837
3600 0.21963787
3800 0.21495533
4000 0.21045978
4200 0.20614094
4400 0.20198917
4600 0.19799584
4800 0.19415258
5000 0.19045155
5200 0.18688555
5400 0.18344784
5600 0.18013185
5800 0.17693168
6000 0.17384155
6200 0.1708563
6400 0.16797084
6600 0.16518043
6800 0.16248076
7000 0.15986751
7200 0.15733679
7400 0.15488493
7600 0.15250836
7800 0.15020382
8000 0.1479681
8200 0.14579828
8400 0.1436916
8600 0.14164531
8800 0.13965698
9000 0.13772428
9200 0.13584487
9400 0.13401668
9600 0.1322376
9800 0.13050577
10000 0.12881935

Hypothesis: 
 [[0.02254861]
 [0.14599545]
 [0.26286733]
 [0.8012864 ]
 [0.95141846]
 [0.9842107 ]] 
Correct (Y): 
 [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  1.0


요약  
H(x) = WX는 이진분류에 적합하지 않음  
따라서 0과 1사이의 값을 돌려주는 함수를 찾은 것이이 시그모이드